# Modelo Item - Item con mejores parametros

In [1]:
import numpy as np
import pandas as pd
import time
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import pickle
import warnings
import gzip
warnings.filterwarnings("ignore")

# Carga Matriz Ratings

In [2]:
%cd D:/Dataset_1/lastfm-dataset-1K/Modelo_Item_Item
#Carga Dataframe Ratings
df_ratings = pd.read_pickle("1.Ratings_Normalizacion_Log.pkl")
df_ratings.rating_lineal = round(df_ratings.rating_lineal)
df_ratings['rating_lineal'] = df_ratings['rating_lineal'].astype('uint8')

D:\Dataset_1\lastfm-dataset-1K\Modelo_Item_Item


# Carga de resultados de evaluación previamente obtenidos

In [3]:
df_resultados_item = pd.read_pickle("1.Evaluacion_Cos_Pearson_Jaccard_ModeloItem_Final.pkl")
#Obtención de mejores parametros para el modelo
df_best_params = df_resultados_item[df_resultados_item.Indice != 'jaccard']
df_best_params = df_best_params.sort_values('RMSE', ascending = True).head(1)

best_k = df_best_params['Tamanio_Vecindario'].values[0]
Umbral = df_best_params['Umbral_Similitud'].values[0]
Medida = df_best_params['Indice'].values[0]

In [4]:
df_best_params

,Tamanio_Vecindario,Umbral_Similitud,Indice,Inicio_seg,Fin_seg,RMSE,Tiempo_Total
8,32,0.7,cosine,26329.812008,29570.290111,0.715442,54.007968


# Aplicación del modelo

In [14]:
reader = Reader( rating_scale = ( 1, 5 ) )
dataset_ratings = Dataset.load_from_df( df_ratings[ [ 'user_id', 'artname', 'rating_lineal']], reader )
# Particionamiento de Dataframe 80% entrenamiento y 20% Validación
train_set, test_set=  train_test_split(dataset_ratings, test_size=.2)
# Configuración de parametros
sim_options = {'name': Medida,'user_based': False , 'min_support': Umbral}
model = KNNBasic(k=best_k, min_k=2, sim_options=sim_options)

#Se le pasa la matriz de utilidad al algoritmo, es decir, el conjunto de entrenamiento
model.fit(trainset=train_set)

# Cálculo de predicción para todos los items del conjunto de validación
test_predictions=model.test(test_set)

#df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.uid, x.iid, x.est) , test_predictions)), columns=labels)
accuracy.rmse( test_predictions, verbose = True )

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.7102


0.7101972567682321

In [6]:
# Dataset de validación 
df_validacion = pd.DataFrame(test_set, columns = ["user_id", "artname", "rating"])
df_validacion.to_pickle("2.Datos_Validacion_ModeloItem.pkl")

# Exportación Modelo

In [21]:
import joblib
joblib.dump(model,filename='3.Model_Item_Item1.pklz')

['3.Model_Item_Item1.pklz']

In [22]:
f = gzip.open('3.Model_Item_Item.pklz','wb')
pickle.dump(model,f, protocol=4)
f.close()

# Prueba Importación y predicción con parametros

In [23]:
#restore the object
f = gzip.open('3.Model_Item_Item.pklz','rb')
Model_UU = pickle.load(f)
result = Model_UU.predict(8, 2843)
f.close()
result

Prediction(uid=8, iid=2843, r_ui=None, est=1.5663272619754651, details={'actual_k': 27, 'was_impossible': False})